In [1]:
from add_embedding import *

In [6]:
### Initial setting
PROJECT_ID = "primeval-argon-420311"
LOCATION = "us-central1"
DIMENSIONS = 768
BUCKET = "embedding-text"
BUCKET_URI = f"gs://{BUCKET}"
DISPLAY_NAME = "text_embedding"
DEPLOYED_INDEX_ID = "text_embedding_endpoint"


llm_type = "gemini" # openai

In [3]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

In [4]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# textembedding-gecko@001
# multimodalembedding

embedding_model = VertexAIEmbeddings(model_name="textembedding-gecko@001")
#vector = embeddings.embed_query("hello, world!")


In [91]:
#NOTE : This operation can take upto 30 seconds
# my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
#     display_name=DISPLAY_NAME,
#     contents_delta_uri=BUCKET_URI,
#     dimensions=DIMENSIONS,
#     approximate_neighbors_count=150,
#     distance_measure_type="DOT_PRODUCT_DISTANCE",
#     index_update_method="STREAM_UPDATE",  # allowed values BATCH_UPDATE , STREAM_UPDATE
# )
# my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
#     display_name=f"{DISPLAY_NAME}_endpoint", public_endpoint_enabled=True
# )

# my_index_endpoint = my_index_endpoint.deploy_index(
#     index=my_index, 
#     deployed_index_id=DEPLOYED_INDEX_ID,
#     min_replica_count=1,
#     max_replica_count=1
# )

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/854115243710/locations/us-central1/indexEndpoints/1709912105005613056
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/854115243710/locations/us-central1/indexEndpoints/1709912105005613056/operations/4246347475935297536
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/854115243710/locations/us-central1/indexEndpoints/1709912105005613056


In [5]:
client = TextEmbeddingClient(index_id="5709952999040745472", endpoint_id = "1709912105005613056")

In [6]:
# Input texts
texts = [
    {"description":"eat pizza for"}
]
# Add vectors and mapped text chunks to your vectore store
client.upload_embedding(texts)

Upserting datapoints MatchingEngineIndex index: projects/854115243710/locations/us-central1/indexes/5709952999040745472
MatchingEngineIndex index Upserted datapoints. Resource name: projects/854115243710/locations/us-central1/indexes/5709952999040745472


['fec0f361-e55d-4a26-9e07-0f7f3dbfbd96']

In [7]:
# Try running a simialarity search
client.similarity_search("pizza")

[Document(page_content='eat pizza for'),
 Document(page_content='eat pizza for')]

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader = PyPDFLoader("https://arxiv.org/pdf/1706.03762.pdf")
pages = loader.load(

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
doc_splits = text_splitter.split_documents(pages)

In [66]:
# Input text with metadata
record_data = [
    {
        "description": "A versatile pair of dark-wash denim jeans."
        "Made from durable cotton with a classic straight-leg cut, these jeans"
        " transition easily from casual days to dressier occasions.",
        "price": 65.00,
        "color": "blue",
        "season": ["fall", "winter", "spring"],
    },
    {
        "description": "A lightweight linen button-down shirt in a crisp white."
        " Perfect for keeping cool with breathable fabric and a relaxed fit.",
        "price": 34.99,
        "color": "white",
        "season": ["summer", "spring"],
    },
    {
        "description": "A soft, chunky knit sweater in a vibrant forest green. "
        "The oversized fit and cozy wool blend make this ideal for staying warm "
        "when the temperature drops.",
        "price": 89.99,
        "color": "green",
        "season": ["fall", "winter"],
    },
    {
        "description": "A classic crewneck t-shirt in a soft, heathered blue. "
        "Made from comfortable cotton jersey, this t-shirt is a wardrobe essential "
        "that works for every season.",
        "price": 19.99,
        "color": "blue",
        "season": ["fall", "winter", "summer", "spring"],
    },
    {
        "description": "A flowing midi-skirt in a delicate floral print. "
        "Lightweight and airy, this skirt adds a touch of feminine style "
        "to warmer days.",
        "price": 45.00,
        "color": "white",
        "season": ["spring", "summer"],
    },
]

In [70]:
from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import (
    Namespace,
    NumericNamespace,
)

In [16]:
# Try running a simple similarity search
# Below code should return 5 results
client.similarity_search("shirt", k=5)

[Document(page_content='eat pizza for')]

In [17]:
# Try running a similarity search with text filter
filters = [Namespace(name="season", allow_tokens=["spring"])]

# Below code should return 4 results now
client.similarity_search("pizza", k=5, filters=filters)

[]

In [73]:
# Initialize the vectore_store as retriever
retriever = vector_store.as_retriever()

In [74]:
# Try running a similarity search with text filter
filters = [Namespace(name="season", allow_tokens=["spring"])]
retriever.search_kwargs = {"filter": filters}

# perform similarity search with filters on retriever
retriever.invoke("What are my options in breathable fabric?")

[Document(page_content='A lightweight linen button-down shirt in a crisp white. Perfect for keeping cool with breathable fabric and a relaxed fit.', metadata={'price': 34.99, 'color': 'white', 'season': ['summer', 'spring']}),
 Document(page_content='A classic crewneck t-shirt in a soft, heathered blue. Made from comfortable cotton jersey, this t-shirt is a wardrobe essential that works for every season.', metadata={'price': 19.99, 'color': 'blue', 'season': ['fall', 'winter', 'summer', 'spring']}),
 Document(page_content='A flowing midi-skirt in a delicate floral print. Lightweight and airy, this skirt adds a touch of feminine style to warmer days.', metadata={'price': 45.0, 'color': 'white', 'season': ['spring', 'summer']}),
 Document(page_content='A versatile pair of dark-wash denim jeans.Made from durable cotton with a classic straight-leg cut, these jeans transition easily from casual days to dressier occasions.', metadata={'price': 65.0, 'color': 'blue', 'season': ['fall', 'winte

In [75]:
from langchain_google_vertexai import VertexAI
llm = VertexAI(model_name="gemini-pro")

In [5]:
from langchain.chains import RetrievalQA
from langchain_google_vertexai import VertexAI

llm = VertexAI(model_name="gemini-pro")

filters = [Namespace(name="season", allow_tokens=["spring"])]
numeric_filters = [NumericNamespace(name="price", value_float=40.0, op="LESS")]

retriever.search_kwargs = {"k": 2, "filter": filters, "numeric_filter": numeric_filters}

retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

question = "What are my options in breathable fabric?"
response = retrieval_qa({"query": question})
print(f"{response['result']}")
print("REFERENCES")
print(f"{response['source_documents']}")

NameError: name 'retriever' is not defined

In [7]:

# Set variables for the current deployed index.
API_ENDPOINT="1754456625.us-central1-854115243710.vdb.vertexai.goog"
INDEX_ENDPOINT="projects/854115243710/locations/us-central1/indexEndpoints/1709912105005613056"
DEPLOYED_INDEX_ID="text_embedding_endpoint"
INDEX_ID = "5709952999040745472"
ENDPOINT_ID = "1709912105005613056"


search_client = VectorSearchClient(index_endpoint=INDEX_ENDPOINT,deployed_index_id=DEPLOYED_INDEX_ID, bucket_uri=BUCKET_URI, index_id=INDEX_ID, endpoint_id = ENDPOINT_I )

In [8]:
embedding_model = VertexAIEmbeddings(model_name="textembedding-gecko@001")
vector = embedding_model.embed_query("pizza")
search_client.find_neighbors(vector)

[[MatchNeighbor(id='9cafdfc2-0cac-4552-be91-08b1dcae35cd', distance=0.7646218538284302, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[]),
  MatchNeighbor(id='fec0f361-e55d-4a26-9e07-0f7f3dbfbd96', distance=0.7646218538284302, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[])]]

In [10]:
search_client.update_index()

Updating MatchingEngineIndex index: projects/854115243710/locations/us-central1/indexes/5709952999040745472


InvalidArgument: 400 The given gcs path `gs://embedding-text/` is not a valid directory or the directory has empty content.